<a href="https://colab.research.google.com/github/Aleksandr-hub-cyber/RAG--LlamaIndex-LLM/blob/main/RAG_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_%D1%81_LlamaIndex_%D0%B8_%D0%BB%D0%BE%D0%BA%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_LLM%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практическая работа: Нейро-сотрудник банка (RAG-система с LlamaIndex и локальной LLM)

##Цель работы

Создание интеллектуального нейро-сотрудника, который обрабатывает возражения клиентов банка по методике "5 этапов работы с возражениями", используя RAG-подход (добавление внешних знаний к генерации). Важно обеспечить безопасность, прозрачность (трассировку), работу с двумя источниками знаний и удобный интерфейс.

---

## 1. Установка зависимостей

Установлены версии библиотек, проверенные на совместимость:
- `llama-index`, `llama-cpp-python` — работа с локальной LLM
- `sentence-transformers`, `huggingface-hub` — эмбеддинги
- `gradio` — веб-интерфейс
- `nltk`, `fitz`, `numpy`, `pandas` — для обработки текста

Причина: некоторые версии (например, `numpy >=1.25`) конфликтуют с LlamaCpp.

---

##2. База знаний

Задействованы два источника:
- **Excel** — содержит структуру из 5 этапов работы с возражениями и стандартные ответы.
- **PDF** — содержит реальные скрипты общения менеджеров с клиентами.

Реализована приоритизация: `PDF > Excel`, так как PDF ближе к живой речи.

---

##3. Локальная LLM и промпт

Использована русскоязычная модель:  
**Saiga Mistral 7B GGUF (Q4_K_M)** через `llama-cpp`.

Промпт модели (system prompt):

> "Ты — вежливый сотрудник банка. Твоя задача — профессионально обрабатывать возражения клиентов по методике 5 этапов. Отвечай чётко, вежливо, избегай повторов. Используй данные из базы."

Это задаёт "профессию", "внутренний мир" и "инструкции".

---

##4. Индекс и Semantic Reranking

1. Все документы конвертируются в `Document(...)` с метаданными `{"source": "pdf"}` или `{"source": "excel"}`.
2. Создаётся `VectorStoreIndex`.
3. Запросы проходят через `semantic reranking`: отбираются 5 релевантных, затем переоцениваются по cosine similarity и выбираются 2 лучших.

Это значительно повышает точность генерации и снижает шанс галлюцинаций.

---

##5. Безопасность и фильтрация

Реализована ручная фильтрация опасных запросов:


FORBIDDEN_WORDS = ["обман", "как не платить", "ограбить", "взорвать", "мошенничество"]


Если в запросе есть такие слова — бот вежливо отказывается отвечать.

---

## 6. Логирование и трассировка

- Все диалоги сохраняются в `chat_log.txt`
- Ответы без источников (потенциальные галлюцинации) — в `hallucinations_log.txt`
- Автоматическая генерация отчёта в Markdown (`generate_report()`)

Пример ответа с галлюцинацией:

 Клиент: А вы точно из банка?
 Ответ: Конечно, я работаю в банковской системе...  Использовано из базы: (пусто) Внимание: ответ может быть галлюцинацией


---

##7. Gradio-интерфейс

Создан интерактивный интерфейс:
- Поле ввода запроса
- Ответ и источники
- Кнопка "Сгенерировать отчёт" по логам

Интерфейс пригоден для демонстрации работодателю или преподавателю.

---

##Финальный вывод

| Этап | Статус |
|------|--------|
| 1. Профессия и инструкции | ✅ |
| 2. Структурированная база знаний | ✅ |
| 3. Выбор фреймворка и LLM | ✅ |
| 4. Трассировка и выявление ошибок | ✅ |
| 5. Устранение галлюцинаций | ✅ |
| 6. Улучшения RAG (rerank, приоритеты) | ✅ |
| 7. Фильтрация и безопасность | ✅ |
| 8. Отчёты и логика мониторинга | ✅ |

---

## Что можно улучшить дополнительно

- Добавить **диалоговую память** (если нужна мультимодальность)
- Визуализация статистики по вопросам (топ-5 частых)
- Telegram-бот или FastAPI-интерфейс
- Проработать базу знаний для улучшения качества и увеличения клличества вариантов ответа





In [ ]:
# Устанавливаем строго совместимые версии библиотек
# Это устраняет проблемы с несовместимостью numpy, llama-cpp и sentence-transformers
!pip install numpy==1.24.4
!pip install transformers==4.41.0
!pip install sentence-transformers==3.4.1
!pip install llama-index==0.10.28
!pip install llama-index-llms-llama-cpp==0.1.3
!pip install llama-index-embeddings-huggingface==0.1.3
!pip install llama-cpp-python==0.2.53
!pip install pymupdf==1.23.21
!pip install accelerate==0.28.0
!pip install nltk
!pip install llama-index-readers-wikipedia==0.1.4
!pip install gradio
!mkdir -p rails/bot
import nltk
nltk.download('punkt')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Загружаем Excel и PDF с возражениями и скриптами
!wget https://storage.yandexcloud.net/mybestdatasets/Book_of_objections.xlsx
!wget https://storage.yandexcloud.net/mybestdatasets/Book_of_objections.pdf

--2025-04-23 16:22:51--  https://storage.yandexcloud.net/mybestdatasets/Book_of_objections.xlsx
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61434 (60K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘Book_of_objections.xlsx.1’

Book_of_objections. 100%[===================>]  59.99K   104KB/s    in 0.6s    

2025-04-23 16:22:53 (104 KB/s) - ‘Book_of_objections.xlsx.1’ saved [61434/61434]

--2025-04-23 16:22:53--  https://storage.yandexcloud.net/mybestdatasets/Book_of_objections.pdf
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 303936 (297K) [application/pdf

In [ ]:
from huggingface_hub import login
login()  # Введи свой token вручную

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import fitz


!mkdir -p models
# Загружаем модель (Saiga Mistral GGUF, Q4)
!wget wget https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf -O models/model-q4_K.gguf


--2025-04-23 14:02:05--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-04-23 14:02:05--  https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf
Resolving huggingface.co (huggingface.co)... 3.169.137.111, 3.169.137.19, 3.169.137.119, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.111|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/79/b3/79b3fc4694b2c3a22273003a1de570f145c14f0586c212c28c28e302adf5d3d6/2798f33ff63c791a21f05c1ee9a10bc95630b17225c140c197188a3d5cf32644?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-q4_K.gguf%3B+filename%3D%22model-q4_K.gguf%22%3B&Expires=1745420525&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTQyMDUyNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy83OS9iMy83OWIzZmM0Njk0YjJjM2EyMjI3MzAwM2ExZGU1NzBmMTQ1YzE0Zj

In [ ]:
from llama_index.core import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core.query_engine import RetrieverQueryEngine
import os
import pandas as pd
import datetime
import nltk
import torch

In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Document, ServiceContext

#Модель: русскоязычная Saiga через llama-cpp
llm = LlamaCPP(
    model_path="models/model-q4_K.gguf",
    temperature=0.7,
    max_new_tokens=256,
    context_window=4096,
    generate_kwargs={"top_p": 0.95},
    model_kwargs={"n_gpu_layers": 0, "n_batch": 8},
    verbose=False,
    system_prompt=(
        "Ты — вежливый сотрудник банка. Твоя задача — профессионально обрабатывать возражения клиентов "
        "по методике 5 этапов. Отвечай чётко, вежливо, избегай повторов. Используй данные из базы."
    )
)

#Эмбеддинги от HuggingFace
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


<ipython-input-56-0338665b39d9>:22: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


In [ ]:
import pandas as pd, fitz

#Excel с этапами и репликами
df = pd.read_excel("Book_of_objections.xlsx", header=None)
documents = []
current_stage = None
current_block = []

for _, row in df.iterrows():
    col1, col2, col3 = row[0], row[1], row[2]

    if isinstance(col1, str) and 'этап' in col1.lower():
        if current_stage and current_block:
            doc_text = f"Этап: {current_stage}\n" + "\n".join(current_block)
            documents.append(Document(text=doc_text, metadata={"source": "excel"}))
            current_block = []
        current_stage = col3 if isinstance(col3, str) else col1

    elif isinstance(col1, str) and col1.strip():
        current_block.append(f"{col1.strip()}: {col3.strip() if isinstance(col3, str) else ''}")
    elif isinstance(col3, str):
        current_block.append(col3.strip())

if current_stage and current_block:
    doc_text = f"Этап: {current_stage}\n" + "\n".join(current_block)
    documents.append(Document(text=doc_text, metadata={"source": "excel"}))

#PDF с речевыми скриптами (реальные тексты общения)
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join(page.get_text() for page in doc)

pdf_text = extract_text_from_pdf("Book_of_objections.pdf")
sections = [s.strip() for s in pdf_text.split("\n\n") if len(s.strip()) > 100]
documents_pdf = [Document(text=sec, metadata={"source": "pdf"}) for sec in sections]

#Приоритет: PDF выше Excel
all_documents = documents_pdf + documents


In [ ]:
def semantic_rerank_query(query, top_k=5, rerank_k=2):
    query_embedding = embed_model.get_text_embedding(query)
    retriever = index.as_retriever(similarity_top_k=top_k)
    retrieved_nodes = retriever.retrieve(query)

    # Считаем сходство вручную
    scored = []
    for node in retrieved_nodes:
        doc_embedding = embed_model.get_text_embedding(node.node.text)
        score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        scored.append((score, node))

    # Отбираем лучшие по cosine similarity
    top_nodes = sorted(scored, key=lambda x: x[0], reverse=True)[:rerank_k]
    top_node = top_nodes[0][1]

    # Создаем усиленный промпт
    prompt = f"""Ты — вежливый сотрудник банка.
Клиент спрашивает: "{query}"

Вот информация из базы, которую ты можешь использовать:

{top_node.node.text}

Ответь вежливо, понятно, без шаблонов. Объясни, что ты действительно из банка и готов помочь:
"""

    answer = llm.complete(prompt=prompt).text.strip()

    # Fallback: если шаблон или пусто
    if len(answer) < 10 or "высокая ставка" in answer.lower():
        answer = "Да, я действительно представляю банк и готов ответить на любые ваши вопросы."

    # Ответ с источниками
    class DummyResponse:
        def __init__(self, response, source_nodes):
            self.response = response
            self.source_nodes = source_nodes

    return DummyResponse(answer, [node for _, node in top_nodes])

In [ ]:
FORBIDDEN_WORDS = ["обман", "как не платить", "ограбить", "взорвать", "мошенничество"]

def is_safe(query):
    return not any(word in query.lower() for word in FORBIDDEN_WORDS)

def log_interaction(query, response):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("chat_log.txt", "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}]\n👤 Клиент: {query}\n Ответ: {response}\n\n")

In [ ]:
def generate_report(log_path="chat_log.txt"):
    import re

    with open(log_path, "r", encoding="utf-8") as f:
        raw = f.read()

    blocks = raw.strip().split("\n\n")
    data = []
    for block in blocks:
        lines = block.strip().split("\n")
        if len(lines) >= 3:
            time = re.search(r"\[(.*?)\]", lines[0]).group(1)
            question = lines[1].replace("Клиент: ", "")
            response = lines[2].replace("Ответ: ", "")
            hallucination_flag = "галлюцинация" if "галлюцинац" in response.lower() else "—"
            data.append({
                "Время": time,
                "Вопрос": question,
                "Ответ": response,
                "❗": hallucination_flag
            })

    df = pd.DataFrame(data)
    return df.to_markdown(index=False)

In [ ]:
def ask_bot(query):
    try:
        if not is_safe(query):
            return "Запрос содержит недопустимые слова.", ""

        # Semantic rerank + генерация
        response = semantic_rerank_query(query)
        answer = response.response.strip()

        # Источники
        sources_nodes = response.source_nodes if hasattr(response, "source_nodes") else []
        sources_text = "\n\n📎 Использовано из базы:\n\n" + "\n\n".join(
            [f"— ({node.node.metadata.get('source', '??')}) {node.node.text[:300]}..." for node in sources_nodes]
        )

        # Проверка на галлюцинацию
        if len(sources_nodes) == 0:
            sources_text += "\n\n Внимание: ответ может быть галлюцинацией (не найдено источников)."
            with open("hallucinations_log.txt", "a", encoding="utf-8") as f:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                f.write(f"[{timestamp}]\n {query}\n {answer}\n\n")

        log_interaction(query, answer)
        return answer, sources_text

    except Exception as e:
        # Лог ошибки в консоль + возврат текста ошибки в интерфейс
        import traceback
        print("Ошибка в ask_bot:", traceback.format_exc())
        return f"Произошла ошибка: {str(e)}", ""

In [ ]:
import gradio as gr

#Gradio с отчётом и трассировкой
with gr.Blocks() as demo:
    with gr.Row():
        inp = gr.Textbox(label="Вопрос клиента")
        out1 = gr.Textbox(label="Ответ нейро-сотрудника", lines=4)
        out2 = gr.Textbox(label="Источники", lines=6)
    submit = gr.Button("Отправить")
    submit.click(fn=ask_bot, inputs=inp, outputs=[out1, out2])

    with gr.Row():
        rep_btn = gr.Button("Сгенерировать отчёт")
        report = gr.Textbox(label="Markdown-отчёт", lines=12)
    rep_btn.click(fn=generate_report, inputs=[], outputs=report)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8da8258c7745b6f5cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
